In [103]:
import pandas as pd
from numpy import argsort
import numpy as np
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

In [104]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Jeevan
[nltk_data]     Reddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jeevan
[nltk_data]     Reddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
npr=pd.read_csv("npr.csv")

In [9]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [10]:
npr["Article"][0]

'In the Washington of 2016, even when the policy can be bipartisan, the politics cannot. And in that sense, this year shows little sign of ending on Dec. 31. When President Obama moved to sanction Russia over its alleged interference in the U. S. election just concluded, some Republicans who had long called for similar or more severe measures could scarcely bring themselves to approve. House Speaker Paul Ryan called the Obama measures ”appropriate” but also ”overdue” and ”a prime example of this administration’s ineffective foreign policy that has left America weaker in the eyes of the world.” Other GOP leaders sounded much the same theme. ”[We have] been urging President Obama for years to take strong action to deter Russia’s worldwide aggression, including its   operations,” wrote Rep. Devin Nunes,  . chairman of the House Intelligence Committee. ”Now with just a few weeks left in office, the president has suddenly decided that some stronger measures are indeed warranted.” Appearing 

In [105]:
stop_words=stopwords.words('english')

In [109]:
npr['pretext']=npr['Article'].apply(lambda x:[item for item in x.split() if item not in stop_words])
npr['pretext']=npr['pretext'].apply(lambda x: ' '.join(x))
npr.head()

,Article,Topic,pretext
0,"In the Washington of 2016, even when the polic...",1,"In Washington 2016, even policy bipartisan, po..."
1,Donald Trump has used Twitter — his prefe...,1,Donald Trump used Twitter — preferred means co...
2,Donald Trump is unabashedly praising Russian...,1,Donald Trump unabashedly praising Russian Pres...
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1,"Updated 2:50 p. m. ET, Russian President Vladi..."
4,"From photography, illustration and video, to d...",2,"From photography, illustration video, data vis..."


In [110]:
lemmatizer = WordNetLemmatizer()

def lemm_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

npr['pretext']=npr['pretext'].apply(lambda text: lemm_words(text))

npr.head()

,Article,Topic,pretext
0,"In the Washington of 2016, even when the polic...",1,"In Washington 2016, even policy bipartisan, po..."
1,Donald Trump has used Twitter — his prefe...,1,Donald Trump used Twitter — preferred mean com...
2,Donald Trump is unabashedly praising Russian...,1,Donald Trump unabashedly praising Russian Pres...
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1,"Updated 2:50 p. m. ET, Russian President Vladi..."
4,"From photography, illustration and video, to d...",2,"From photography, illustration video, data vis..."


In [111]:
from sklearn.feature_extraction.text import CountVectorizer

In [130]:
cv=CountVectorizer(strip_accents = 'unicode',
                   stop_words="english",
                                lowercase = True,
                                max_df = 0.9, 
                                min_df = 2)

In [131]:
#cv=CountVectorizer(max_df=0.9,min_df=2,stop_words='english')

In [132]:
dtm=cv.fit_transform(npr["pretext"])
dtm.shape

(11992, 53285)

In [133]:
from sklearn.decomposition import LatentDirichletAllocation

In [134]:
# for TF DTM
lda_cv = LatentDirichletAllocation(n_components=7, random_state=42)
lda_cv.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

In [135]:

pyLDAvis.lda_model.prepare(lda_cv, dtm,cv)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.085654 -0.027205       1        1  21.127027
4     -0.131292 -0.098499       2        1  20.918762
2      0.005112  0.044212       3        1  15.045453
6     -0.078019 -0.132580       4        1  12.795249
3     -0.114897  0.117407       5        1  11.823132
5      0.021798  0.148229       6        1  10.764634
0      0.211644 -0.051563       7        1   7.525744, topic_info=            Term          Freq         Total Category  logprob  loglift
49016      trump  22443.000000  22443.000000  Default  30.0000  30.0000
22066     health   8540.000000   8540.000000  Default  29.0000  29.0000
37081  president  11634.000000  11634.000000  Default  28.0000  28.0000
9537     clinton   7295.000000   7295.000000  Default  27.0000  27.0000
41860       says  21857.000000  21857.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
33107        npr    552.138688   6533.118321   Topic7  -6.4127   0.1160
41860       says    583.507110  21857.253419   Topic7  -6.3574  -1.0364
5492        best    514.738722   3525.189730   Topic7  -6.4828   0.6628
52593       work    525.469427   8855.327717   Topic7  -6.4622  -0.2376
51756        way    520.543097  11504.175355   Topic7  -6.4716  -0.5088

[664 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
1          1  0.013511   000
1          2  0.138067   000
1          3  0.344982   000
1          4  0.087171   000
1          5  0.277429   000
...      ...       ...   ...
52918      4  0.114726  year
52918      5  0.150262  year
52918      6  0.093907  year
52918      7  0.090255  year
53205      6  0.999519  zika

[2047 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 7, 4, 6, 1])

In [136]:
#Grab the vocabulary of words

In [137]:
# Grab the topics

In [138]:
for i, topic in enumerate(lda_cv.components_):
    print(f"The Top 20 words for Topic {i}")
    print([cv.get_feature_names_out()[index] for index in topic.argsort()[-20:]])
    print("\n")
    print("\n")

The Top 20 words for Topic 0
['american', 'art', 'say', 'prince', 'museum', 'day', 'record', 'make', 'sound', 'artist', 'world', 'time', 'band', 'like', 'food', 'year', 'album', 'new', 'song', 'music']




The Top 20 words for Topic 1
['year', 've', 'going', 'new', 'says', 'book', 'woman', 'make', 'story', 'don', 'life', 'really', 'say', 'thing', 'way', 'time', 'know', 'think', 'people', 'like']




The Top 20 words for Topic 2
['000', 'education', 'make', 'work', 'day', 'world', 'family', 'home', 'city', 'new', 'country', 'time', 'child', 'like', 'student', 'year', 'people', 'school', 'says', 'say']




The Top 20 words for Topic 3
['make', 'program', 'medical', 'plan', 'new', 'tax', 'patient', 'insurance', 'study', 'state', 'drug', 'year', 'company', 'said', 'says', 'percent', 'care', 'say', 'people', 'health']




The Top 20 words for Topic 4
['department', 'clinton', 'year', 'federal', 'white', 'campaign', 'administration', 'people', 'new', 'law', 'country', 'obama', 'court', 'gove

In [139]:
topic_result=lda_cv.transform(dtm)

In [140]:
npr['Topic']=topic_result.argmax(axis=1)

In [142]:
npr=npr.drop('pretext',axis=1)
npr

,Article,Topic
0,"In the Washington of 2016, even when the polic...",4
1,Donald Trump has used Twitter — his prefe...,4
2,Donald Trump is unabashedly praising Russian...,4
3,"Updated at 2:50 p. m. ET, Russian President Vl...",4
4,"From photography, illustration and video, to d...",2
...,...,...
11987,The number of law enforcement officers shot an...,6
11988,"Trump is busy these days with victory tours,...",4
11989,It’s always interesting for the Goats and Soda...,5
11990,The election of Donald Trump was a surprise to...,4
